# CV loop


In [20]:
%matplotlib inline
import numpy as np
import sys
import os

caffe_root = '../../../caffe/'
sys.path.insert(0, caffe_root + 'python')
import caffe
from caffe import layers as L, params as P

import h5py

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
np_loaded_data_file = '../data/train_data_cleaned.npz'
if not os.path.isfile(np_loaded_data_file):
    print "%s does not exist. See facial_recog_kaggle.ipynb" % np_loaded_data_file
else:
    print "loading %s" % np_loaded_data_file
    npzfile = np.load(np_loaded_data_file)
    print "loaded: ", npzfile.files
    X_train_clean_cv, y_train_clean_cv = npzfile['X_train_clean_cv'], npzfile['y_train_clean_cv']
    X_val_clean_cv, y_val_clean_cv = npzfile['X_val_clean_cv'], npzfile['y_val_clean_cv']
    feature_labels = npzfile['feature_labels']
    print X_train_clean_cv.shape, y_train_clean_cv.shape
    print X_val_clean_cv.shape, y_val_clean_cv.shape


f = h5py.File('../data/train_data_lenet.hd5', 'w')
f.create_dataset('data', data=X_train_clean_cv, compression='gzip', compression_opts=4)
f.create_dataset('label', data=y_train_clean_cv, compression='gzip', compression_opts=4)
f.close()

f = h5py.File('../data/test_data_lenet.hd5', 'w')
f.create_dataset('data', data=X_val_clean_cv, compression='gzip', compression_opts=4)
f.create_dataset('label', data=y_val_clean_cv, compression='gzip', compression_opts=4)
f.close()

loading ../data/train_data_cleaned.npz
loaded:  ['X_val_clean_cv', 'y_val_clean_cv', 'y_train_clean_cv', 'feature_labels', 'X_train_clean_cv']
(1640, 1, 96, 96) (1640, 30)
(500, 1, 96, 96) (500, 30)


In [ ]:
num_epochs = 50
td_size = X_train_clean_cv.shape[0]
# td_size = 50
val_data_size = X_val_clean_cv.shape[0]
num_labels = y_train_clean_cv.shape[1]

num_cv_points = 10

batch_size_vec = np.random.choice([32, 64, 96, 128], num_cv_points, replace=True)

base_learning_rate_exps = np.random.uniform(-3, -6, num_cv_points)
base_learning_rate = np.power(10, base_learning_rate_exps)

reg_param_exps = np.random.uniform(-1, -6, num_cv_points)
reg_param_vec = np.power(10, reg_param_exps)

print zip(batch_size_vec, base_learning_rate, reg_param_vec)

[(96, 2.6774287305930625e-05, 1.2685843885889934e-06), (96, 2.6452968002151156e-06, 0.001499378173512326), (64, 0.00047761349214236841, 0.0042614849125783266), (128, 1.6272285424801794e-06, 4.1919335954933465e-05), (96, 2.0233359032726703e-05, 7.7271364406277302e-05), (96, 1.2119310762665393e-05, 0.00056575337039514858), (96, 9.9542320282277079e-06, 0.041471819316919727), (96, 0.00014221530264473372, 0.000484055829283041), (32, 3.5441959061272491e-05, 0.00035792800395328066), (96, 0.00058964999302223966, 0.091607222356769158)]


In [ ]:
from net_archs import lenet
from def_solver import create_solver

train_net_path = 'train_net.prototxt'
test_net_path = 'test_net.prototxt'
solver_config_fname_base = 'lenet_solver'

results = {}
for batch_size, base_lr, reg_param in zip(batch_size_vec, base_learning_rate, reg_param_vec):
    
    train_suffix = "__b_lr_%.02e__reg_param_%0.02e__batch_size_%d" % (base_lr, reg_param, batch_size)
    print "train_suffix = %s" % train_suffix
    
    with open(train_net_path, 'w') as f:
        f.write(str(lenet(hdf5_list='../data/train_hdf5.list', batch_size=batch_size)))

    with open(test_net_path, 'w') as f:
        f.write(str(lenet(hdf5_list='../data/test_hdf5.list', batch_size=batch_size)))
    
    s = create_solver(base_lr, reg_param, train_net_path, test_net_path)
    solver_config_fname = solver_config_fname_base + train_suffix + ".prototxt"
    with open(solver_config_fname, 'w') as f:
        f.write(str(s))
    
    solver = None
    solver = caffe.get_solver(solver_config_fname)
    
    num_iter_per_epoch = int(np.ceil(float(td_size) / batch_size))
    niter = num_iter_per_epoch * num_epochs

    niter_val = int(np.ceil( float(val_data_size) / batch_size))
    val_interval = num_iter_per_epoch

    print "batch_size = %d, base_learning_rate = %e, reg_param = %e" % (batch_size, base_lr, reg_param)
    
    print "niter = %d, val_interval = %d" % (niter, val_interval)

    train_error = np.zeros(niter)
    val_error = np.zeros( int(np.ceil(float(niter) / val_interval)))

    for it in range(niter):
        solver.step(1)

        train_error[it] = solver.net.blobs['loss'].data

        if (it % val_interval) == 0:

            val_error_this = 0
            for test_it in range(niter_val):
                solver.test_nets[0].forward()
                val_error_this += solver.test_nets[0].blobs['loss'].data / niter_val

            val_error[it // val_interval] = val_error_this

            print "it = %d, train_error = %e, val_error = %e" % (it, train_error[it], val_error[it // val_interval])
            
        results[(batch_size, base_lr, reg_param)] = (train_error, val_error)
        
        # save model
        model_save_name = "lenet" + train_suffix + ".caffemodel"
        solver.net.save(model_save_name)
    